In [15]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import math

In [16]:
df = pd.read_csv('https://raw.githubusercontent.com/murpi/wilddata/master/quests/spotify.zip')

In [17]:
def popularity_score(popularity):
  popularity = popularity / 20
  popularity = math.ceil(popularity)
  return popularity

In [18]:
df.drop('time_signature', axis = 1, inplace = True)
df = df[(df['genre'] != 'Soundtrack') & (df['genre'] != 'Movie')]
df = df[(df['duration_ms'] > 90000) & (df['duration_ms'] < 900000)]
df['genre'].replace("Children’s Music", "Children's Music", inplace = True)
top_songs = df[df['popularity'] >= 80]
top_scaled = top_songs.copy()
scaler = MinMaxScaler()
top_scaled['loudness_scaled'] = scaler.fit_transform(top_songs['loudness'].values.reshape(-1, 1))

In [19]:
top_scaled.to_csv('data/top_polar.csv.zip', compression='zip')

In [20]:
df_music_dummies = pd.concat([df , df['genre'].str.get_dummies()], axis = 1)
df_music_dummies['mode'] = df_music_dummies['mode'].factorize()[0]
df_music_dummies_keys = pd.concat([df_music_dummies , df_music_dummies['key'].str.get_dummies()], axis = 1)
df_music_dummies_keys["popularity_score"] = df["popularity"].apply(popularity_score)
cols = ['genre', 'artist_name', 'track_name', 'track_id', 'key']
df_music_dummies_keys.drop(cols, axis=1, inplace=True)
df_music_without_0_pop = df_music_dummies_keys[df_music_dummies_keys['popularity_score'] != 0]

In [27]:
df_final = df_music_without_0_pop.copy()
df_final['tempo'].apply(round)
df_final = df_final.astype({'tempo': 'int'})
df_final.drop(columns = ['duration_ms'], axis = 1, inplace = True)

In [32]:
df_final.to_csv('data/music_ml.csv.zip', compression='zip')

In [28]:
df_final['tempo'].min()

32

In [31]:
df_final['tempo'].max()

242

In [33]:
df_final

,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo,...,C,C#,D,D#,E,F,F#,G,G#,popularity_score
135,65,0.08300,0.724,0.689,0.000000,0.3040,-5.922,0,0.1350,146,...,0,0,1,0,0,0,0,0,0,4
136,63,0.32300,0.685,0.610,0.000000,0.1020,-5.221,0,0.0439,94,...,1,0,0,0,0,0,0,0,0,4
137,62,0.06750,0.762,0.520,0.000004,0.1140,-5.237,0,0.0959,75,...,0,0,0,0,0,1,0,0,0,4
138,61,0.36000,0.563,0.366,0.002430,0.0955,-6.896,0,0.1210,85,...,0,0,0,0,0,0,0,0,0,4
139,68,0.59600,0.653,0.621,0.000000,0.0811,-5.721,0,0.0409,100,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232720,39,0.00384,0.687,0.714,0.544000,0.0845,-10.626,1,0.0316,115,...,0,0,1,0,0,0,0,0,0,2
232721,38,0.03290,0.785,0.683,0.000880,0.2370,-6.944,0,0.0337,113,...,0,0,0,0,1,0,0,0,0,2
232722,47,0.90100,0.517,0.419,0.000000,0.0945,-8.282,1,0.1480,84,...,0,0,1,0,0,0,0,0,0,3
232723,44,0.26200,0.745,0.704,0.000000,0.3330,-7.137,1,0.1460,100,...,0,0,0,0,0,0,0,0,0,3
